In [7]:
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image
import paddle
import os
from PIL import Image
import json

ocr = PaddleOCR(use_angle_cls=True, lang='ch', use_gpu=False, ocr_version='PP-OCRv4')
img_path = 'D:\Snipaste_2024-11-06_14-19-53.png'
image = Image.open(img_path).convert('RGB')
result = ocr.ocr(img_path, cls=False)

result = result[0]
boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
scores = [line[1][1] for line in result]
im_show = draw_ocr(image, boxes, txts, scores)
im_show = Image.fromarray(im_show)


[2024/11/11 14:43:29] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\superfqr/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\superfqr/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batc

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Snipaste_2024-11-06_14-19-53.png'

In [3]:
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image, ImageEnhance
import paddle
import os
from natsort import natsorted
import json
import cv2
import numpy as np

ocr = PaddleOCR(use_angle_cls=True, lang='ch', use_gpu=True,
                ocr_version='PP-OCRv3'
                )
result_dir = 'ocr\\python_test\\datasets\\cali_set_det'

download https://paddleocr.bj.bcebos.com/PP-OCRv3/chinese/ch_PP-OCRv3_det_infer.tar to C:\Users\superfqr/.paddleocr/whl\det\ch\ch_PP-OCRv3_det_infer\ch_PP-OCRv3_det_infer.tar


100%|██████████| 3740/3740 [00:07<00:00, 493.15it/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv3/chinese/ch_PP-OCRv3_rec_infer.tar to C:\Users\superfqr/.paddleocr/whl\rec\ch\ch_PP-OCRv3_rec_infer\ch_PP-OCRv3_rec_infer.tar


100%|██████████| 11630/11630 [00:12<00:00, 967.51it/s] 

[2024/11/14 11:30:06] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\superfqr/.paddleocr/whl\\det\\ch\\ch_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\superfqr/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch

[2024/11/14 11:30:13] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2024/11/14 11:30:15] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0


In [3]:
# 强标注
img_dir = 'datasets\\cali_set_det'
threshold = 0.6
sort_img_dir = natsorted(os.listdir(img_dir))
train_data = {}

for filename in sort_img_dir:
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(img_dir, filename)
        try:         
            result = ocr.ocr(img_path, cls=True)
            if result is None or len(result) == 0:
                print(f"文件 {filename} 的 OCR 结果为空，跳过该文件。")
                continue

            entries = []
            for line in result[0]:
                box = line[0]
                text = line[1][0]
                score = line[1][1]
                illegibility = score < threshold
                if illegibility:
                     continue
                entries.append({
                    "illegibility": illegibility,
                    "points": box,
                    "score": score,
                    "transcription": text
                })

            train_data[f"{os.path.splitext(filename)[0]}"] = entries
            

        except Exception as e:
            print(f"处理文件 {filename} 时出错: {e}")

# 保存为 JSON 文件
with open('full_labels.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

[2024/11/13 18:39:14] ppocr DEBUG: dt_boxes num : 3, elapsed : 4.303012371063232
[2024/11/13 18:39:14] ppocr DEBUG: cls num  : 3, elapsed : 0.01661372184753418
[2024/11/13 18:39:14] ppocr DEBUG: rec_res num  : 3, elapsed : 0.019434690475463867
[2024/11/13 18:39:14] ppocr DEBUG: dt_boxes num : 7, elapsed : 0.11151838302612305
[2024/11/13 18:39:14] ppocr DEBUG: cls num  : 7, elapsed : 0.022369384765625
[2024/11/13 18:39:14] ppocr DEBUG: rec_res num  : 7, elapsed : 0.04711508750915527
[2024/11/13 18:39:14] ppocr DEBUG: dt_boxes num : 4, elapsed : 0.141343355178833
[2024/11/13 18:39:14] ppocr DEBUG: cls num  : 4, elapsed : 0.014626264572143555
[2024/11/13 18:39:14] ppocr DEBUG: rec_res num  : 4, elapsed : 0.019709348678588867
[2024/11/13 18:39:14] ppocr DEBUG: dt_boxes num : 7, elapsed : 0.10573673248291016
[2024/11/13 18:39:14] ppocr DEBUG: cls num  : 7, elapsed : 0.01318979263305664
[2024/11/13 18:39:15] ppocr DEBUG: rec_res num  : 7, elapsed : 0.045516252517700195
[2024/11/13 18:39:15] 

In [81]:
# 弱标注

img_dir = "datasets\\cali_set_rec"
sort_img_dir = natsorted(os.listdir(img_dir))
train_data = {}
threshold = 0.6
for filename in sort_img_dir:
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(img_dir, filename)
        try:         
            result = ocr.ocr(img_path, cls=True)
            if result is None or len(result) == 0:
                print(f"文件 {filename} 的 OCR 结果为空，跳过该文件。")
                continue

            # 只提取置信度最高的识别结果
            best_result = result[0][0]  # 假设结果数组的第一项为最佳识别
            text = best_result[1][0]  # 识别的文本
            score = best_result[1][1]  # 置信度
            entries = [{
                "score": score,
                "transcription": text
            }]
            
            train_data[f"{os.path.splitext(filename)[0]}"] = entries
            
        except Exception as e:
            print(f"处理文件 {filename} 时出错: {e}")

# 保存为 JSON 文件
with open('weak_labels.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

[2024/11/13 00:03:53] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.1199805736541748
[2024/11/13 00:03:53] ppocr DEBUG: cls num  : 1, elapsed : 0.006528377532958984
[2024/11/13 00:03:53] ppocr DEBUG: rec_res num  : 1, elapsed : 0.09181690216064453
[2024/11/13 00:03:53] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.15020513534545898
[2024/11/13 00:03:53] ppocr DEBUG: cls num  : 1, elapsed : 0.006684780120849609
[2024/11/13 00:03:53] ppocr DEBUG: rec_res num  : 1, elapsed : 0.01993536949157715
[2024/11/13 00:03:53] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.01818370819091797
[2024/11/13 00:03:53] ppocr DEBUG: cls num  : 1, elapsed : 0.006455659866333008
[2024/11/13 00:03:53] ppocr DEBUG: rec_res num  : 1, elapsed : 0.015642166137695312
[2024/11/13 00:03:53] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.018032550811767578
[2024/11/13 00:03:53] ppocr DEBUG: cls num  : 1, elapsed : 0.0050585269927978516
[2024/11/13 00:03:53] ppocr DEBUG: rec_res num  : 1, elapsed : 0.012392044067382812
[2024/11/13 0

In [80]:

img_dir = 'datasets\\cali_set_det'
result_dir = 'results'  # 保存结果图片的文件夹
os.makedirs(result_dir, exist_ok=True)  # 如果文件夹不存在则创建
threshold = 0.7

# 按文件名排序
sort_img_dir = natsorted(os.listdir(img_dir))

for filename in sort_img_dir:
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(img_dir, filename)
        try:         
            result = ocr.ocr(img_path, cls=True)
            if result is None or len(result) == 0:
                print(f"文件 {filename} 的 OCR 结果为空，跳过该文件。")
                continue

            # 打开图像
            image = Image.open(img_path).convert('RGB')
            result_img_path = os.path.join(result_dir, f'result_{filename}')

            # 处理 OCR 结果，根据 threshold 设置文本内容
            boxes = [line[0] for line in result[0]]
            txts = [line[1][0] if line[1][1] >= threshold else "###" for line in result[0]]
            scores = [line[1][1] for line in result[0]]

            # 绘制 OCR 结果并保存图片
            im_show = draw_ocr(image, boxes, txts, scores)
            im_show = Image.fromarray(im_show)        
            im_show.save(result_img_path)

        except Exception as e:
            print(f"处理文件 {filename} 时出错: {e}")


[2024/11/13 00:02:49] ppocr DEBUG: dt_boxes num : 3, elapsed : 0.4721641540527344
[2024/11/13 00:02:49] ppocr DEBUG: cls num  : 3, elapsed : 0.019515514373779297
[2024/11/13 00:02:49] ppocr DEBUG: rec_res num  : 3, elapsed : 0.021310091018676758
[2024/11/13 00:02:49] ppocr DEBUG: dt_boxes num : 7, elapsed : 0.10777139663696289
[2024/11/13 00:02:49] ppocr DEBUG: cls num  : 7, elapsed : 0.016592979431152344
[2024/11/13 00:02:49] ppocr DEBUG: rec_res num  : 7, elapsed : 0.04526567459106445
[2024/11/13 00:02:49] ppocr DEBUG: dt_boxes num : 4, elapsed : 0.13899636268615723
[2024/11/13 00:02:49] ppocr DEBUG: cls num  : 4, elapsed : 0.009563446044921875
[2024/11/13 00:02:49] ppocr DEBUG: rec_res num  : 4, elapsed : 0.019628047943115234
[2024/11/13 00:02:49] ppocr DEBUG: dt_boxes num : 7, elapsed : 0.10902190208435059
[2024/11/13 00:02:49] ppocr DEBUG: cls num  : 7, elapsed : 0.014559268951416016
[2024/11/13 00:02:49] ppocr DEBUG: rec_res num  : 7, elapsed : 0.0434873104095459
[2024/11/13 00:0

In [78]:
import os
from PIL import Image, ImageEnhance
from paddleocr import PaddleOCR

# 定义图像处理和OCR处理的函数
def process_image(image_path, output_dir):
    try:
        # 读取图像
        img = Image.open(image_path)

        # 增强对比度
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(2.0)

        # 进行二值化处理
        img = img.point(lambda p: p > 128 and 255)

        # 获取文件名
        filename = os.path.basename(image_path)

        # 保存处理后的图像
        output_path = os.path.join(output_dir, filename)
        img.save(output_path)

        # 如果需要可以打印或处理OCR的结果
        if result:
            print(f"识别结果：{result}")
        else:
            print(f"未能识别 {filename} 的文本。")
        
        return result

    except Exception as e:
        print(f"处理文件 {image_path} 时出错: {e}")
        return None

# 设定源目录和目标目录
input_dir = 'datasets/cali_set_det'
output_dir = 'datasets/process_cali_set_det'

# 确保输出目录存在
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 遍历输入目录中的所有图片文件
for filename in os.listdir(input_dir):
    if filename.endswith(('.jpg', '.png', '.jpeg')):  # 只处理图片文件
        img_path = os.path.join(input_dir, filename)
        
        # 处理每张图片并进行OCR识别
        process_image(img_path, output_dir)

识别结果：[[[[[290.0, 689.0], [306.0, 689.0], [306.0, 709.0], [290.0, 709.0]], ('铺', 0.6077750325202942)]]]
识别结果：[[[[[290.0, 689.0], [306.0, 689.0], [306.0, 709.0], [290.0, 709.0]], ('铺', 0.6077750325202942)]]]
识别结果：[[[[[290.0, 689.0], [306.0, 689.0], [306.0, 709.0], [290.0, 709.0]], ('铺', 0.6077750325202942)]]]
识别结果：[[[[[290.0, 689.0], [306.0, 689.0], [306.0, 709.0], [290.0, 709.0]], ('铺', 0.6077750325202942)]]]
识别结果：[[[[[290.0, 689.0], [306.0, 689.0], [306.0, 709.0], [290.0, 709.0]], ('铺', 0.6077750325202942)]]]
识别结果：[[[[[290.0, 689.0], [306.0, 689.0], [306.0, 709.0], [290.0, 709.0]], ('铺', 0.6077750325202942)]]]
识别结果：[[[[[290.0, 689.0], [306.0, 689.0], [306.0, 709.0], [290.0, 709.0]], ('铺', 0.6077750325202942)]]]
识别结果：[[[[[290.0, 689.0], [306.0, 689.0], [306.0, 709.0], [290.0, 709.0]], ('铺', 0.6077750325202942)]]]
识别结果：[[[[[290.0, 689.0], [306.0, 689.0], [306.0, 709.0], [290.0, 709.0]], ('铺', 0.6077750325202942)]]]
识别结果：[[[[[290.0, 689.0], [306.0, 689.0], [306.0, 709.0], [290.0, 709.0]],